In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import os

print(tf.__version__)

1.12.0


In [2]:
# 导入 MNIST data
mnist = input_data.read_data_sets("D:/Study/python/jupyter/DL/TF/Tutorial/wk04/MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/Study/python/jupyter/DL/TF/Tutorial/wk04/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting D:/Study/python/jupyter/DL/TF/Tutorial/wk04/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting D:/Study/python/jupyter/DL/TF/Tutorial/wk04/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting D:/Study/python/jupyter/DL/TF/Tutorial/wk04/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 设定超参数
learning_rate = 0.001
num_steps = 50
batch_size = 64
display_step = 10

# 设定网络保存的路径
model_path = "/tmp/model.ckpt"
# if not os.path.exists(model_path):
#     os.makedirs(model_path)


# 网络参数的设定
num_input=784 # 28*28
num_classes =10 # label的类别
dropout = 0.75 #

# 输入数据
X = tf.placeholder(tf.float32,[None,num_input])
Y = tf.placeholder(tf.float32,[None,num_classes])
keep_prob = tf.placeholder(tf.float32)

In [4]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [5]:
# 创建辅助函数
def conv2d(x,W,b,strides=1):
    x = tf.nn.conv2d(x,W,\
                     strides=[1,strides,strides,1],\
                     padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,\
                         ksize=[1,k,k,1],\
                         strides=[1,k,k,1],\
                         padding='SAME')
# 创建模型
def conv_net(x,weight,biases,dropout):
    # 将784个一维的feature 转换成
    # 4—D:[batch_size,Height,width,channel]
    x = tf.reshape(x,shape=[-1,28,28,1])
    
    #第一层CNN
    conv1 = conv2d(x,weights['wc1'],biases['bc1'])
    conv1 = maxpool2d(conv1,k=2)
    
    #第二层CNN
    conv2 = conv2d(conv1,weights['wc2'],biases['bc2'])
    conv2 = maxpool2d(conv2,k=2)
    
    #全连接层 Fully connected
    fc1 = tf.reshape(conv2,[-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1,dropout)
    
    out = tf.add(tf.matmul(fc1,weights['out']),biases['out'])
    
    return out

In [6]:
#构建模型
logits = conv_net(X,weights,biases,keep_prob)
prediction = tf.nn.softmax(logits)

#定义损失函数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=logits,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
saver = tf.train.Saver()

In [8]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                      Y: mnist.test.labels[:256],
                                      keep_prob: 1.0}))
    
    save_path = saver.save(sess,model_path)
    print("Model saved in file: %s" % save_path)

Step 1, Minibatch Loss= 56800.3203, Training Accuracy= 0.062
Step 10, Minibatch Loss= 24015.4805, Training Accuracy= 0.312
Step 20, Minibatch Loss= 14828.9805, Training Accuracy= 0.422
Step 30, Minibatch Loss= 11296.7842, Training Accuracy= 0.547
Step 40, Minibatch Loss= 4630.2266, Training Accuracy= 0.734
Step 50, Minibatch Loss= 3610.7129, Training Accuracy= 0.766
Optimization Finished!
Testing Accuracy: 0.7734375
Model saved in file: /tmp/model.ckpt
